# Exercícios sobre Classificadores Bayesianos

### Autor: Bruno Oliveira Marques

Inicialmente é preciso inserir as variáveis correspondentes às probabilidades já conhecidas:

In [44]:
# coding: utf-8
import pandas as pd

# probabilidade de navegador corresponder ao perfil
P_N = pd.Series([0.4, 0.6], index=['N=1', 'N=2'])

# probabilidade de navegador realizar compras ou não
P_C_N = pd.DataFrame({'V': [0.9, 0.3], 'F': [0.1, 0.7]}, index=['N=1', 'N=2'])

P_DuTd_N = pd.DataFrame({  # probabilidades de acesso para cada tipo de navegador
    'Du=V,Td=V': [0.3, 0.3],
    'Du=V,Td=F': [0.1, 0.3],
    'Du=F,Td=V': [0.2, 0.3],
    'Du=F,Td=F': [0.4, 0.1],
}, index=['N=1', 'N=2'])

A partir daí, podemos aplicar os classificadores bayesianos.

Para o classificador pela máxima probabilidade ``a posteriori``, é necessário calcular as probabilidades máximas para cada tipo de acesso:

In [45]:
P_N_DuTd = P_DuTd_N.apply(lambda col: col*P_N, axis=0)

hmax = P_N_DuTd.idxmax(axis=0)
hmax.name = 'h_max'

P_C_hmax = P_C_N.loc[hmax]
P_C_hmax.index = hmax.index

cmap = P_C_hmax.idxmax(axis=1)
cmap.name = 'C_MAP'

pd.concat([P_N_DuTd.transpose(), hmax, P_C_hmax, cmap], axis=1)

,N=1,N=2,h_max,F,V,C_MAP
"Du=F,Td=F",0.16,0.06,N=1,0.1,0.9,V
"Du=F,Td=V",0.08,0.18,N=2,0.7,0.3,F
"Du=V,Td=F",0.04,0.18,N=2,0.7,0.3,F
"Du=V,Td=V",0.12,0.18,N=2,0.7,0.3,F


Com isso temos os valores para a coluna C_MAP.

O classificador ótimo bayesiano usa uma fórmula similar:

In [47]:
P_C_V = P_N_DuTd.apply(lambda col: col*P_C_N['V'], axis=0).sum(axis=0)
P_C_F = P_N_DuTd.apply(lambda col: col*P_C_N['F'], axis=0).sum(axis=0)
P_Cj = pd.concat([P_C_V, P_C_F], axis=1, keys=['V', 'F'])

copt = P_Cj.idxmax(axis=1)
copt.name = 'C_OPT'

pd.concat([P_Cj, copt], axis=1)

,V,F,C_OPT
"Du=F,Td=F",0.162,0.058,V
"Du=F,Td=V",0.126,0.134,F
"Du=V,Td=F",0.090,0.130,F
"Du=V,Td=V",0.162,0.138,V


E, assim, também obtivemos C_OPT.

O resultado final é o seguinte:


In [48]:
pd.concat([cmap, copt], axis=1)

,C_MAP,C_OPT
"Du=F,Td=F",V,V
"Du=F,Td=V",F,F
"Du=V,Td=F",F,F
"Du=V,Td=V",F,V
